### Ingest races.csv file

In [0]:
# Create a text widget for data source input with an empty default value
dbutils.widgets.text("p_data_source", "")

# Retrieve the value of the data source widget
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
# Create a text widget for file date input with a default value of "2021-03-21"
dbutils.widgets.text("p_file_date", "2021-03-21")

# Retrieve the value of the file date widget
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# Running the configuration notebook to load necessary configurations and settings
%run "../includes/configuration"

In [0]:
# Run the common functions notebook to make its functions available in the current notebook
%run "../includes/common_functions"

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [0]:
# Importing necessary classes from pyspark.sql.types to define schema for DataFrame
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
# Define schema for the races DataFrame
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) 
])

In [0]:
# Read the races CSV file into a DataFrame with the specified schema and header option
races_df = spark.read \
    .option("header", True) \
    .schema(races_schema) \
    .csv(f"{raw_folder_path}/{v_file_date}/races.csv")

##### Step 2 - Add ingestion date and race_timestamp to the dataframe

In [0]:
# Import necessary functions from pyspark.sql.functions for timestamp conversion and column operations
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [0]:
# Add a new column 'race_timestamp' by combining 'date' and 'time' columns and converting to timestamp
# Add a new column 'data_source' with a constant value from variable v_data_source
# Add a new column 'file_date' with a constant value from variable v_file_date
races_with_timestamp_df = races_df.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
# Add a new column 'ingestion_date' with the current timestamp to the DataFrame
races_with_ingestion_date_df = add_ingestion_date(races_with_timestamp_df)

##### Step 3 - Select only the columns required & rename as required

In [0]:
# Select and rename specific columns from the DataFrame
races_selected_df = races_with_ingestion_date_df.select(
    col('raceId').alias('race_id'), 
    col('year').alias('race_year'), 
    col('round'), 
    col('circuitId').alias('circuit_id'),
    col('name'), 
    col('ingestion_date'), 
    col('race_timestamp')
)

##### Write the output to processed container in parquet format

In [0]:
# Write the DataFrame to a Delta table, partitioned by 'race_year', with overwrite mode
races_selected_df.write.mode("overwrite").partitionBy('race_year').format("delta").saveAsTable("f1_processed.races")

In [0]:
%sql
-- Select all columns from the races table in the f1_processed schema
SELECT * FROM f1_processed.races;

In [0]:
# Exit the notebook with a success message
dbutils.notebook.exit("Success")